In [ ]:
import numpy as np
# import scipy.io as scio
import pandas as pd

import Accuracy_motion_state

import VFC_dyad
import Idiff
import Idiff_sit

# 输入数据路径
GROUP1_DATA2021_FOLDER = 'data2021/Class1-Parent'#亲子组
GROUP2_DATA2021_FOLDER = 'data2021/Class2-Stranger'#陌生人组
GROUP1_DATA2023_FOLDER = 'data2023/Class1-Parent'#亲子组
GROUP2_DATA2023_FOLDER = 'data2023/Class2-Stranger'#陌生人组
GROUP1_DATAALL_FOLDER = 'data_all/Class1-Parent'#亲子组
GROUP2_DATAALL_FOLDER = 'data_all/Class2-Stranger'#陌生人组

# 输入数据结构：
# 每个数据文件包含6种拼图场景，其中前面的1/2表示儿童拼或者成人拼，
# 后面的1/2/3表示单独/陪伴/合作,只关注3（合作）
sit_c1 = 'ca11' #儿童拼-单独
sit_a1 = 'ca21' #成人拼-单独
sit_c2 = 'ca12' #儿童拼-陪伴
sit_a2 = 'ca22' #成人拼-陪伴
sit_c3 = 'ca13' #儿童拼-合作
sit_a3 = 'ca23' #成人拼-合作
# 每种情况包含4个cell文件，分别对应4个频段：1-3，3-6，6-9，9-12
delta,theta,alpha,belta=0,1,2,3
freq_all=['delta','theta','alpha','belta']
# 每个cell文件的格式为通道数*时间点数*被试数
# 通道数: 其中1-28为儿童，29-56为成人，通过subc选择类型
# 28个通道及其对应的索引为：
channame_full = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','T7','T8','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_all = list(range(0,28))
# 26个通道及其对应的索引为：
channame_test = ['Fp1','Fp2','Fz','F3','F4','F7','F8','FC1','FC2','FC5','FC6','Cz','C3','C4','CP1','CP2','CP5','CP6','Pz','P3','P4','P7','P8','Oz','O1','O2']
id_chan_test = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

# 参数设置
chan_num = 28 #通道数
edge_num = int(chan_num*(chan_num-1)/2) #边数
Nuptran_indices = VFC_dyad.uptran_mask(chan_num) #矩阵上三角列向量的位置
Euptran_indices = VFC_dyad.uptran_mask(edge_num) #矩阵上三角列向量的位置
Vtype='uptran'

# itype指边连接强度的计算方式，'p','c','d'分别代表：PLV复指数、corr相关、delta做差
itype_all=['e','c','d']
itype=itype_all[0]
# corr_type指边与边之间共波动特性的计算方式，'cos','pearson','delta'分别代表：
# 余弦相似度，皮尔逊相关系数，直接做差
corr_type_all=['cos','pearson','delta']
corr_type=corr_type_all[0]

tp=2000

freq=theta #频段
subc='child'  #对象类型 

condition = 'motion'

In [10]:
if condition == 'motion':
    GROUP_DATA_FOLDER = GROUP1_DATAALL_FOLDER
    motion_comp = [sit_a3,sit_c3]
elif condition == 'state':
    GROUP_DATA_FOLDER = GROUP1_DATA2021_FOLDER
    motion_comp = [sit_c1,sit_c2,sit_c3]


## 1、神经活动（包括拼图动作和状态）识别性，总体Idiff

### 计算nFC方法的结果，用以比较

In [11]:
acrcy_list_nfc,binary_acrcy_list_nfc=Accuracy_motion_state.Acc_motion_nFC_all(GROUP_DATA_FOLDER=GROUP_DATA_FOLDER,motion_comp=motion_comp,
                    freq=freq,subc=subc,corr_type=corr_type)

0.8049105727805744 0.7859908088838947
0.7116986699037994 0.687414827193377
0.7214330033515294 0.719805320950855
0.811733375891962 0.8070675319700762
0.7680481149725488 0.7642273971566886
0.7784647838610286 0.7913335029069715
0.7529819519198538 0.7694554182837049
0.8040952503683095 0.8022003098525748
0.7585629911358718 0.7447784736700176
0.7231933975455315 0.7384930040378167
0.7727252380734747 0.7576396903597653
0.7395626420043635 0.7232469258286789
0.8271986956992295 0.7988426285961772
0.7470976262337959 0.7527864737182927
0.7918135028560277 0.7766431737140347
0.7630285879126907 0.7801713561174003
0.7432612901934779 0.7072509202825118
0.7103182312761687 0.7139647749360555


In [12]:
acrcy_mean_nfc = np.mean(acrcy_list_nfc)
acrcy_mean_nfc

0.358351504591342

In [13]:
binary_acrcy_mean_nfc = np.mean(binary_acrcy_list_nfc)
binary_acrcy_mean_nfc

0.546265545189505

### 方法：使用最小生成树MST对VeFC序列矩阵降维

In [14]:
acrcy_list_efc,binary_acrcy_list_efc=Accuracy_motion_state.Acc_motion_eFC_all(GROUP_DATA_FOLDER,motion_comp,freq,subc,corr_type,None)
acrcy_mean_efc = np.mean(acrcy_list_efc)
acrcy_mean_efc

0.12835099916011583 0.11106622921968336
0.27761646459089573 0.20933956921591423
0.17414076540615686 0.1819253433505459
0.15037670792857247 0.12570493553311396
0.22146260860305578 0.20557200027030387
0.12859544196155223 0.1858138349614167
0.15389899389712947 0.1792835439173661
0.19756510754287096 0.18509237925990246
0.18676187681463677 0.13188298058434014
0.16275717452099248 0.1794600367765801
0.1734215167217359 0.14573989527786524
0.23484094732807628 0.14428997411809658
0.23640333106770134 0.22229545398054731
0.08648818831389232 0.1428098020025457
0.3028798625814418 0.21090805521594808
0.11847120038083277 0.14969824674644636
0.22955300362774503 0.197694145252456
0.20682916145246544 0.07554981078936003


0.3728587233130514

In [15]:
binary_acrcy_mean_efc = np.mean(binary_acrcy_list_efc)
binary_acrcy_mean_efc

0.5568439197444458

In [16]:
acrcy_list = np.c_[acrcy_list_nfc, acrcy_list_efc]
df_acrcy_list = pd.DataFrame(acrcy_list)

# 添加列名和索引
df_acrcy_list.columns=['nfc','efc'] #设置列名
df_acrcy_list

,nfc,efc
0,0.475000,0.325000
1,0.463415,0.341463
2,0.333333,0.416667
3,0.230769,0.461538
4,0.634146,0.439024
5,0.268293,0.341463
6,0.190476,0.309524
7,0.275000,0.400000
8,0.256410,0.410256
9,0.275000,0.275000


In [17]:
output_excel = 'output/Acc_'+condition+'.xlsx'
# df=pd.DataFrame() #构造原始数据文件
# df.to_excel(output_excel)

# 创建一个新的工作表，命名为：场景+对象类型+频段
new_sheet_name = subc+'-'+freq_all[freq] 
# # 比较各个分支的结果
# new_sheet_name = itype+'-'+corr_type
excel_writer = pd.ExcelWriter(output_excel, mode='a',engine='openpyxl',if_sheet_exists='replace')

# 保存至excel
df_acrcy_list.to_excel(excel_writer, sheet_name=new_sheet_name)
excel_writer._save()

FileNotFoundError: [Errno 2] No such file or directory: 'output/Acc_state.xlsx'